FUNCIONES REQUERIDAS PARA LA API


In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

Extracción de los conjuntos de datos: 


In [3]:
# Se extrean los datos de df_games que contiene información relacionada a los juegos que tiene la plataforma,asi como df_items que 
#contiene la información relacionada al consumo de los juegos por parte de los usuarios y  df_reviews que contiene la información 
#relacionada a los usuarios que realizaron review de los juegos que consumen.

df_games = pq.read_table('df_outputsteam_games.parquet').to_pandas()
df_reviews = pq.read_table('df_user_reviews.parquet').to_pandas()
df_items = pq.read_table('df_user_items_explode.parquet').to_pandas()


1.  #Cantidad de dinero gastado por usuario y cantidad de items consumidos


In [4]:

# Se extraen las columnas 'items_count', 'user_id' e 'item_id'
df_gastos_items = df_items[['items_count', 'user_id', 'item_id']]
df_gastos_items

,items_count,user_id,item_id
0,277,76561197970982479,10
1,277,76561197970982479,20
2,277,76561197970982479,30
3,277,76561197970982479,40
4,277,76561197970982479,50
...,...,...,...
5170009,7,76561198329548331,346330
5170010,7,76561198329548331,373330
5170011,7,76561198329548331,388490
5170012,7,76561198329548331,521570


In [5]:
#Extrayendo del df_games los precios de los juegos
precio_juegos = df_games[['price', 'id']]
# Elimina los duplicados
previo_juegos = precio_juegos.drop_duplicates(subset='id', keep='first')
# Se renombra 'id' por 'item_id' para unir mas adelante
precio_juegos = precio_juegos.rename(columns={'id':'item_id'})
precio_juegos

,price,item_id
88310,4.99,761140
88311,0.00,643980
88312,0.00,670290
88313,0.99,767400
88315,3.99,772540
...,...,...
120439,1.99,745400
120440,1.99,773640
120441,4.99,733530
120442,1.99,610660


In [6]:
# Uniendo los 2 dataframe auxiliares para conformar un dataframe final llamado df_gastos_items

df_gastos_items = df_gastos_items.merge(precio_juegos, on='item_id', how='left')
df_gastos_items

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
5094087,7,76561198329548331,346330,0.00
5094088,7,76561198329548331,373330,NaN
5094089,7,76561198329548331,388490,0.00
5094090,7,76561198329548331,521570,0.00


In [7]:
#se observa valores faltantes en la columna price
df_gastos_items[df_gastos_items['price'].isnull()]


,items_count,user_id,item_id,price
13,277,76561197970982479,34440,NaN
21,277,76561197970982479,340,NaN
26,277,76561197970982479,9340,NaN
27,277,76561197970982479,228200,NaN
37,277,76561197970982479,17340,NaN
...,...,...,...,...
5094076,177,76561198326700687,508380,NaN
5094078,177,76561198326700687,510660,NaN
5094082,177,76561198326700687,519170,NaN
5094088,7,76561198329548331,373330,NaN


In [8]:
# se asume que son gratuitos y se rellenan con cero 
df_relleno = df_gastos_items['price'].fillna(0.0)
# Se borra la columna original y se concatena la columna rellena con todo el dataframe
df_gastos_items = pd.concat([df_gastos_items.drop('price', axis=1), df_relleno], axis=1)
df_gastos_items

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
5094087,7,76561198329548331,346330,0.00
5094088,7,76561198329548331,373330,0.00
5094089,7,76561198329548331,388490,0.00
5094090,7,76561198329548331,521570,0.00


In [9]:
#Finalmente, se elimina la columna 'item_id' y se agrupa por usuario para sumar la cantidad gastada por usuario.

# Se elimina 'item_id'
df_gastos_items = df_gastos_items.drop('item_id', axis=1)
df_gastos_items.columns

Index(['items_count', 'user_id', 'price'], dtype='object')

In [10]:
# Se agrupa por usuario sumando los precios de los juegos consumidos
df_gastos_items_group = df_gastos_items.groupby('user_id')['price'].sum().reset_index()
df_gastos_items_group

,user_id,price
0,--000--,397.779999
1,--ace--,166.819992
2,--ionex--,99.930000
3,-2SV-vuLB-Kg,427.500000
4,-404PageNotFound-,1509.319946
...,...,...
70907,zzonci,19.980000
70908,zzoptimuszz,64.980003
70909,zzydrax,99.940002
70910,zzyfo,828.510010


In [11]:
# dataframe auxiliar 'df_count_items' conteniendo la cantidad de items por usuario.
df_count_items = df_gastos_items[['items_count', 'user_id']]
# se eliminan los duplicados
df_count_items = df_count_items.drop_duplicates(subset='user_id', keep='first')
df_count_items

,items_count,user_id
0,277,76561197970982479
277,888,js41637
1165,137,evcentric
1302,328,Riot-Punch
1630,541,doctr
...,...,...
5093561,321,76561198320136420
5093882,4,ArkPlays7
5093886,22,76561198323066619
5093908,177,76561198326700687


In [12]:
# Se juntan los dos dataframe auxiliares 'df_gastos_items_group' que contiene el gasto de cada usuario con 'df_count_items' 
#que contiene la cantidad de items consumidos por cada usuario.

df_gastos_items = df_count_items.merge(df_gastos_items_group, on='user_id', how='right')
df_gastos_items

,items_count,user_id,price
0,58,--000--,397.779999
1,44,--ace--,166.819992
2,23,--ionex--,99.930000
3,68,-2SV-vuLB-Kg,427.500000
4,149,-404PageNotFound-,1509.319946
...,...,...,...
70907,5,zzonci,19.980000
70908,61,zzoptimuszz,64.980003
70909,13,zzydrax,99.940002
70910,84,zzyfo,828.510010


2. Cantidad de items y contenido free por empresa desarrolladora:


In [13]:
price_anio_developer = df_games[['price', 'release_date', 'developer', 'id']]
# Se renombra la columna 'id' para unirla con el dataframe anterior
price_anio_developer = price_anio_developer.rename(columns={'id':'item_id'})
# se eliminan los duplicados
df_items_developer = price_anio_developer.drop_duplicates()
df_items_developer

,price,release_date,developer,item_id
88310,4.99,2018,Kotoshiro,761140
88311,0.00,2018,Secret Level SRL,643980
88312,0.00,2017,Poolians.com,670290
88313,0.99,2017,彼岸领域,767400
88315,3.99,2018,Trickjump Games Ltd,772540
...,...,...,...,...
120439,1.99,2018,Bidoniera Games,745400
120440,1.99,2018,"Nikita ""Ghost_RUS""",773640
120441,4.99,2018,Sacada,733530
120442,1.99,2018,Laush Dmitriy Sergeevich,610660


In [27]:
dfs = [df_gastos_items, df_items_developer]
# Nombres correspondientes a cada DataFrame
names = ['df_gastos_items', 'df_items_developer']

for df, name in zip(dfs, names):
    archivo = f'data/{name}_unido.csv'
    df.to_csv(archivo, index=False, encoding='utf-8')
    print(f"DataFrame '{name}' guardado como '{archivo}'")


DataFrame 'df_gastos_items' guardado como 'data/df_gastos_items_unido.csv'
DataFrame 'df_items_developer' guardado como 'data/df_items_developer_unido.csv'
